In [3]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model (pretrained on COCO dataset)
model = YOLO('yolov8n.pt')  # Using YOLOv8n (nano) model

# Define the video file paths
input_video_path = 'Traffic2.mp4'
output_video_path = 'out2.mp4'

# Open the input video file
cap = cv2.VideoCapture(input_video_path)

# Get the width and height of the frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)

    # Extract detections from the first frame result
    detections = results[0].boxes.data.cpu().numpy()  # Extract detection results

    # Draw bounding boxes and labels on the frame
    for det in detections:
        x1, y1, x2, y2, conf, cls = det[:6]
        label = model.names[int(cls)]
        
        # Define conditions for encroachments and blockages
        if label in ['car', 'truck', 'bus', 'motorbike']:  # Adjust based on your needs
            # Check for encroachments at the sides of the road
            if x1 < frame_width * 0.1 or x2 > frame_width * 0.9:
                color = (0, 0, 255)  # Red for encroachments
                text = f'{label} (encroachment)'
            # Check for vehicles blocking the road (middle of the frame)
            elif y1 < frame_height * 0.4 and y2 > frame_height * 0.6:
                color = (0, 255, 255)  # Yellow for blockages
                text = f'{label} (blockage)'
            else:
                color = (0, 255, 0)  # Green for normal detections
                text = f'{label}'
        elif label in ['person']:  # Include other specific encroachments
            color = (0, 255, 0)  # Red for encroachments
            text = f'{label} '
        elif label in ['parking_meter']:  # Temporary obstacles
            color = (255, 0, 0)  # Blue for temporary obstacles
            text = f'{label} (temporary obstacle)'
        else:
            color = (0, 255, 0)  # Green for normal detections
            text = f'{label}'

        # Draw bounding box and label
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Write the frame into the output video
    out.write(frame)

    # Optional: Display the resulting frame
    # cv2.imshow('Frame', frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release everything when the job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video processing complete. Output saved to", output_video_path)



0: 384x640 12 persons, 2 cars, 1 motorcycle, 1 bus, 3 trucks, 3 umbrellas, 99.8ms
Speed: 3.0ms preprocess, 99.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 car, 1 motorcycle, 1 bus, 2 trucks, 2 umbrellas, 111.6ms
Speed: 3.0ms preprocess, 111.6ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 motorcycle, 2 buss, 3 trucks, 2 umbrellas, 106.7ms
Speed: 3.0ms preprocess, 106.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 cars, 1 bus, 3 trucks, 3 umbrellas, 95.4ms
Speed: 2.0ms preprocess, 95.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 motorcycle, 1 bus, 3 trucks, 2 umbrellas, 88.2ms
Speed: 2.0ms preprocess, 88.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 bus, 3 trucks, 2 umbrellas, 86.3ms
Speed: 3.0ms preprocess, 86.3ms inference